<a href="https://colab.research.google.com/github/GeorgeTelles/Detector_Triangulos/blob/main/Detector_de_Padr%C3%A3o_Gr%C3%A1fico_Triangulo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://raw.githubusercontent.com/GeorgeTelles/georgetelles/f69531ec6b293b5148563588a764c010015d315e/logo_clara.png" width="300" align="left"/>
<img src="https://raw.githubusercontent.com/GeorgeTelles/georgetelles/f69531ec6b293b5148563588a764c010015d315e/logo_dark.png" width="300" align="left"/>

---
# **Detector de Padrões gráficos com Python: Triangulos**

#### Detecção do padrão de triângulo nos preços
---

Padrões gráficos nos preços referem-se a certos **movimentos recorrentes e identificáveis** nos preços de ativos.<p>
Esses padrões são baseados na **interação entre oferta e demanda** no mercado para buscar antecipar os movimentos futuros nos preços.<p>
São classificados como padrões de **continuação ou reversão**.

O padrão **Triângulo** é formado pela convergência das linhas de suporte e resistência, criando uma área triangular no gráfico.



https://www.investopedia.com/terms/t/triangle.asp

<img src="https://www.investopedia.com/thmb/OsZtPqhYt4nOMbXsbhmx5nejkW0=/750x0/filters:no_upscale():max_bytes(150000):strip_icc():format(webp)/UnderstandingTriangle2-0651c3c900b3422cadc70d83555a5072.png" width="600" align="left"/>

**Triângulo Ascendente**: linha de resistência horizontal e uma linha de suporte inclinada ascendente.<p>
Indica que a pressão de compra está aumentando gradualmente enquanto a pressão de venda é contida.<p>
É classificado como uma formação de continuação, ou seja, após a consolidação no triângulo, espera-se que a tendência anterior continue.<p>
Entretanto, se ocorre a falha no rompimento, muitos acreditam que funciona como um forte padrão de reversão.

# 1. Bibliotecas

In [ ]:
# Manipulação de dados e séries temporais
import numpy as np
import pandas as pd
from datetime import datetime, timedelta, date, time
from time import sleep

# Análises estatísticas
from scipy import stats

# Dados OHLCV
import yfinance as yf

# Visualização
import plotly.graph_objects as go
import mplfinance as mpf

# 2. Baixar os dados OHLC: 1 ativo

Determinar data inicial e final dos dados e o ativo desejado

In [ ]:
start_date = datetime(2023,1,1)
end_date = datetime.now() + timedelta(days=1)

In [ ]:
ticker = "AALR3.SA"

Baixar os dados com o MT5

In [ ]:
df = yf.download(ticker,start=start_date, end=end_date)
df = pd.DataFrame(df)
df = df.drop(['Volume'], axis=1)
df.insert(0, 'ticker', ticker)
df.columns = df.columns.str.lower().str.replace(' ', '_')

# 3. Criar as funções para localizar os micro pivôs (micro topos e fundos)

In [ ]:
def pivots_high_idx(df, i, n_cdls):
    ls_pivots_high = []
    for x in range(n_cdls):
        ls_pivots_high.append(df.high.iloc[i] > df.high.iloc[i-(x+1)])
        ls_pivots_high.append(df.high.iloc[i] >= df.high.iloc[i+(x+1)])
    if all(ls_pivots_high):
        return 1
    else:
        return 0

def pivots_low_idx(df, i, n_cdls):
    ls_pivots_low = []
    for x in range(n_cdls):
        # Use .iloc to access by position
        ls_pivots_low.append(df.low.iloc[i] < df.low.iloc[i-(x+1)])
        ls_pivots_low.append(df.low.iloc[i] <= df.low.iloc[i+(x+1)])
    if all(ls_pivots_low):
        return -1
    else:
        return 0

In [ ]:
n_cdls=3
ls_pivots_high = []
ls_pivots_low = []
ls_pivots_high.extend(np.zeros(n_cdls, dtype=int))
ls_pivots_low.extend(np.zeros(n_cdls, dtype=int))

for i in range(n_cdls, len(df)-n_cdls):
    ls_pivots_high.append(pivots_high_idx(df, i, n_cdls))
    ls_pivots_low.append(pivots_low_idx(df, i, n_cdls))

ls_pivots_high.extend(np.zeros(n_cdls, dtype=int))
ls_pivots_low.extend(np.zeros(n_cdls, dtype=int))

df['pivots_high_idx'] = ls_pivots_high
df['pivots_low_idx'] = ls_pivots_low

In [ ]:
def pivots_high_values(df, i):
    if df.pivots_high_idx.iloc[i] == 1:
        return df['high'].iloc[i] + 0.01
    else:
        return np.NaN

def pivots_low_values(df, i):
    if df.pivots_low_idx.iloc[i] == -1:
        return df['low'].iloc[i] - 0.01
    else:
        return np.NaN


In [ ]:
ls_pivots_high_values = []
ls_pivots_low_values = []

for i in range(len(df)):
    ls_pivots_high_values.append(pivots_high_values(df, i))
    ls_pivots_low_values.append(pivots_low_values(df, i))

df['pivots_high_values'] = ls_pivots_high_values
df['pivots_low_values'] = ls_pivots_low_values

In [ ]:
fig_window=500
pivots_high = mpf.make_addplot(df.pivots_high_values[:fig_window],type='scatter',markersize=15,marker='^', color='blue')
pivots_low = mpf.make_addplot(df.pivots_low_values[:fig_window],type='scatter',markersize=15,marker='^', color='red')
mpf.plot(df[:fig_window], type='candle', addplot=[pivots_high, pivots_low], figsize=(10,6))

# 4. Regressão linear dos pivôs

In [ ]:
backward_cdls = 50  # Quantos candles no máximo formam um triângulo
count_patterns = 0

for i in range(backward_cdls, len(df)):

    if df.pivots_high_idx.iloc[i] == 1 or df.pivots_low_idx.iloc[i] == -1:
        df_temporary = df.iloc[i-backward_cdls:i+1].reset_index()
        df_temporary_high = df_temporary[df_temporary.pivots_high_idx == 1]
        df_temporary_low = df_temporary[df_temporary.pivots_low_idx == -1]

        if len(df_temporary_high) > 3 and len(df_temporary_low) > 3:
            regr_test_high = stats.linregress(df_temporary_high.index, df_temporary_high.high)
            regr_test_low = stats.linregress(df_temporary_low.index, df_temporary_low.low)

            if (-0.2 < regr_test_high.rvalue < 0.2) and (regr_test_low.rvalue > 0.8): #é aqui que eu regulo a inclinação das retas
                print("data:", df_temporary.iloc[-1]['Date'].strftime('%Y-%m-%d'),'candle_idx =', i, 'rvalue_high =', regr_test_high.rvalue, 'rvalue_low =', regr_test_low.rvalue)
                count_patterns += 1

                fig = go.Figure(data=[go.Candlestick(x=df_temporary.index,
                                                    open=df_temporary['open'],
                                                    high=df_temporary['high'],
                                                    low=df_temporary['low'],
                                                    close=df_temporary['close'],
                                                    name=df_temporary.ticker.iloc[0])])

                fig.data[0].increasing.fillcolor = 'white'
                fig.data[0].increasing.line.color = 'black'
                fig.data[0].increasing.line.width = 1.5
                fig.data[0].decreasing.fillcolor = 'black'
                fig.data[0].decreasing.line.color = 'black'
                fig.data[0].decreasing.line.width = 1.5

                fig.add_scatter(x=df_temporary.index, y=df_temporary['pivots_high_values'], mode="markers",
                                marker=dict(size=7, color="green"))

                fig.add_scatter(x=df_temporary.index, y=df_temporary['pivots_low_values'], mode="markers",
                                marker=dict(size=7, color="red"),
                                name="pivot_low")
                fig.add_trace(go.Scatter(x=df_temporary_high.index, y=(regr_test_high.slope * df_temporary_high.index + regr_test_high.intercept),
                                            mode='lines', name='max slope', line=dict(color='green', width=3)))
                fig.add_trace(go.Scatter(x=df_temporary_low.index, y=(regr_test_low.slope * df_temporary_low.index + regr_test_low.intercept),
                                            mode='lines', name='min slope', line=dict(color='red', width=3)))
                fig.update_layout(xaxis_rangeslider_visible=False, width=600, height=400, showlegend=False)

                fig.show()
print('Triângulos encontrados =', count_patterns)
